In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [6]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    ################################################ s', Q = env(s, a)
    ################################################ ~s', ~Q = env(s, ~a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, 
                                                                   labels=next_states_labels))
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels))
    eQs_logits = tf.reshape(eQs, shape=[-1])
    aQs_logits = tf.reshape(aQs, shape=[-1])
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    _, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                  action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
                                                                     labels=tf.ones_like(rates))) # max
    ###################################################### Q(s,a)= r + Q'(s',a') # max
    ###################################################### ~Q(s,~a)= r # min
    ###################################################### ~Q(s,~a)= r + Q'(s',a') # max
    targetQs = rewards + (gamma * aQs2_logits)
    eloss += tf.reduce_mean(tf.square(eQs_logits - targetQs)) # real
    eloss += tf.reduce_mean((aQs_logits+aQs2_logits)/2) # min
    aloss2 += -tf.reduce_mean((aQs_logits+aQs2_logits)/2) # max
    return actions_logits, aloss, eloss, aloss2

In [7]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [8]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [9]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [10]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [11]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [12]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [13]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:15.0000 R:15.0000 rate:0.0300 aloss:0.7389 eloss:3.0248 aloss2:1.9455 exploreP:0.9985
Episode:1 meanR:18.0000 R:21.0000 rate:0.0420 aloss:0.7251 eloss:3.0529 aloss2:1.9267 exploreP:0.9964
Episode:2 meanR:17.3333 R:16.0000 rate:0.0320 aloss:0.6867 eloss:3.1421 aloss2:1.8413 exploreP:0.9949
Episode:3 meanR:18.7500 R:23.0000 rate:0.0460 aloss:0.6970 eloss:3.1357 aloss2:1.8500 exploreP:0.9926
Episode:4 meanR:20.4000 R:27.0000 rate:0.0540 aloss:0.7230 eloss:3.0678 aloss2:1.9106 exploreP:0.9900
Episode:5 meanR:23.1667 R:37.0000 rate:0.0740 aloss:0.6884 eloss:3.1930 aloss2:1.8026 exploreP:0.9863
Episode:6 meanR:22.1429 R:16.0000 rate:0.0320 aloss:0.6881 eloss:3.1236 aloss2:1.8727 exploreP:0.9848
Episode:7 meanR:23.2500 R:31.0000 rate:0.0620 aloss:0.7127 eloss:3.0247 aloss2:1.9503 exploreP:0.9818
Episode:8 meanR:22.1111 R:13.0000 rate:0.0260 aloss:0.6842 eloss:3.0576 aloss2:1.9631 exploreP:0.9805
Episode:9 meanR:23.6000 R:37.0000 rate:0.0740 aloss:0.6944 eloss:3.1817 aloss2:1.8

Episode:80 meanR:23.7778 R:22.0000 rate:0.0440 aloss:0.6918 eloss:1.1293 aloss2:3.7189 exploreP:0.8266
Episode:81 meanR:23.6707 R:15.0000 rate:0.0300 aloss:0.6937 eloss:1.1158 aloss2:3.7128 exploreP:0.8253
Episode:82 meanR:23.7108 R:27.0000 rate:0.0540 aloss:0.6928 eloss:1.0210 aloss2:4.0031 exploreP:0.8231
Episode:83 meanR:23.7262 R:25.0000 rate:0.0500 aloss:0.6939 eloss:0.9947 aloss2:3.9997 exploreP:0.8211
Episode:84 meanR:23.6000 R:13.0000 rate:0.0260 aloss:0.6925 eloss:0.9933 aloss2:3.9277 exploreP:0.8201
Episode:85 meanR:23.6977 R:32.0000 rate:0.0640 aloss:0.6930 eloss:0.9447 aloss2:4.1358 exploreP:0.8175
Episode:86 meanR:23.6322 R:18.0000 rate:0.0360 aloss:0.6935 eloss:0.9096 aloss2:4.1261 exploreP:0.8160
Episode:87 meanR:23.6136 R:22.0000 rate:0.0440 aloss:0.6938 eloss:0.8833 aloss2:4.1356 exploreP:0.8142
Episode:88 meanR:23.5393 R:17.0000 rate:0.0340 aloss:0.6925 eloss:0.8818 aloss2:4.1977 exploreP:0.8129
Episode:89 meanR:23.4667 R:17.0000 rate:0.0340 aloss:0.6941 eloss:0.8068 

Episode:159 meanR:20.1200 R:9.0000 rate:0.0180 aloss:0.7084 eloss:-0.6761 aloss2:7.2366 exploreP:0.7146
Episode:160 meanR:20.0000 R:11.0000 rate:0.0220 aloss:0.6939 eloss:-0.6633 aloss2:6.9707 exploreP:0.7138
Episode:161 meanR:19.9400 R:16.0000 rate:0.0320 aloss:0.6770 eloss:-0.3180 aloss2:7.3713 exploreP:0.7127
Episode:162 meanR:19.9500 R:13.0000 rate:0.0260 aloss:0.7095 eloss:-0.6518 aloss2:7.4518 exploreP:0.7118
Episode:163 meanR:19.9800 R:16.0000 rate:0.0320 aloss:0.6927 eloss:-0.7535 aloss2:7.2237 exploreP:0.7106
Episode:164 meanR:19.7900 R:14.0000 rate:0.0280 aloss:0.7178 eloss:-0.3696 aloss2:7.2660 exploreP:0.7097
Episode:165 meanR:19.4500 R:18.0000 rate:0.0360 aloss:0.7077 eloss:-0.7396 aloss2:7.5109 exploreP:0.7084
Episode:166 meanR:19.5500 R:26.0000 rate:0.0520 aloss:0.7061 eloss:-0.7119 aloss2:7.4003 exploreP:0.7066
Episode:167 meanR:19.2700 R:12.0000 rate:0.0240 aloss:0.7026 eloss:-0.6996 aloss2:7.3613 exploreP:0.7058
Episode:168 meanR:19.1800 R:17.0000 rate:0.0340 aloss:0.

Episode:237 meanR:31.3400 R:82.0000 rate:0.1640 aloss:0.8365 eloss:-2.3523 aloss2:17.5846 exploreP:0.5441
Episode:238 meanR:31.7000 R:59.0000 rate:0.1180 aloss:0.8335 eloss:-1.7999 aloss2:17.8062 exploreP:0.5410
Episode:239 meanR:32.3900 R:87.0000 rate:0.1740 aloss:0.8098 eloss:-2.4443 aloss2:18.2600 exploreP:0.5364
Episode:240 meanR:32.4900 R:26.0000 rate:0.0520 aloss:0.8321 eloss:-2.2402 aloss2:18.4703 exploreP:0.5350
Episode:241 meanR:32.5000 R:15.0000 rate:0.0300 aloss:0.8113 eloss:-2.0073 aloss2:18.6030 exploreP:0.5342
Episode:242 meanR:32.8700 R:54.0000 rate:0.1080 aloss:0.8284 eloss:-1.6102 aloss2:18.5596 exploreP:0.5314
Episode:243 meanR:34.3700 R:163.0000 rate:0.3260 aloss:0.8516 eloss:-1.8762 aloss2:18.8045 exploreP:0.5230
Episode:244 meanR:34.4500 R:22.0000 rate:0.0440 aloss:0.8874 eloss:-2.4060 aloss2:18.9187 exploreP:0.5218
Episode:245 meanR:34.1200 R:12.0000 rate:0.0240 aloss:0.7741 eloss:-2.6932 aloss2:18.8865 exploreP:0.5212
Episode:246 meanR:35.3400 R:135.0000 rate:0.2

Episode:314 meanR:115.3600 R:25.0000 rate:0.0500 aloss:0.7446 eloss:-4.4948 aloss2:35.1263 exploreP:0.2016
Episode:315 meanR:115.5600 R:33.0000 rate:0.0660 aloss:0.7805 eloss:-4.6560 aloss2:34.7789 exploreP:0.2010
Episode:316 meanR:115.2200 R:35.0000 rate:0.0700 aloss:0.7635 eloss:-4.7183 aloss2:35.0906 exploreP:0.2003
Episode:317 meanR:114.9800 R:23.0000 rate:0.0460 aloss:0.7793 eloss:-4.0109 aloss2:35.2049 exploreP:0.1999
Episode:318 meanR:114.9500 R:22.0000 rate:0.0440 aloss:0.7331 eloss:-3.3205 aloss2:35.6327 exploreP:0.1995
Episode:319 meanR:114.4200 R:24.0000 rate:0.0480 aloss:0.8094 eloss:-3.9189 aloss2:35.4525 exploreP:0.1990
Episode:320 meanR:114.1400 R:25.0000 rate:0.0500 aloss:0.8169 eloss:-3.5467 aloss2:35.8373 exploreP:0.1985
Episode:321 meanR:114.0300 R:31.0000 rate:0.0620 aloss:0.7718 eloss:-5.0923 aloss2:35.5947 exploreP:0.1980
Episode:322 meanR:114.0600 R:17.0000 rate:0.0340 aloss:0.8146 eloss:-4.4538 aloss2:35.7775 exploreP:0.1976
Episode:323 meanR:112.8700 R:23.0000 

...

Episode:1792 meanR:9.7200 R:8.0000 rate:0.0160 aloss:0.8286 eloss:-4.2966 aloss2:30.4799 exploreP:0.0341
Episode:1793 meanR:9.7300 R:10.0000 rate:0.0200 aloss:0.7793 eloss:-3.0787 aloss2:31.1377 exploreP:0.0341
Episode:1794 meanR:9.7400 R:10.0000 rate:0.0200 aloss:0.8344 eloss:-0.9698 aloss2:30.4140 exploreP:0.0341
Episode:1795 meanR:9.7500 R:10.0000 rate:0.0200 aloss:0.8288 eloss:-4.7256 aloss2:30.0312 exploreP:0.0340
Episode:1796 meanR:9.7700 R:11.0000 rate:0.0220 aloss:0.8198 eloss:-4.0147 aloss2:30.5587 exploreP:0.0340
Episode:1797 meanR:9.7900 R:11.0000 rate:0.0220 aloss:0.8295 eloss:-2.5578 aloss2:30.7222 exploreP:0.0340
Episode:1798 meanR:9.8100 R:10.0000 rate:0.0200 aloss:0.8286 eloss:-4.8889 aloss2:30.1863 exploreP:0.0340
Episode:1799 meanR:9.8300 R:10.0000 rate:0.0200 aloss:0.7919 eloss:-2.9624 aloss2:30.3732 exploreP:0.0339
Episode:1800 meanR:9.8200 R:9.0000 rate:0.0180 aloss:0.7610 eloss:-1.6494 aloss2:30.4936 exploreP:0.0339
Episode:1801 meanR:9.8000 R:10.0000 rate:0.0200 

Episode:1870 meanR:9.6500 R:10.0000 rate:0.0200 aloss:0.7944 eloss:-3.0850 aloss2:29.6358 exploreP:0.0324
Episode:1871 meanR:9.6600 R:11.0000 rate:0.0220 aloss:0.7723 eloss:-1.2583 aloss2:29.8822 exploreP:0.0323
Episode:1872 meanR:9.6600 R:10.0000 rate:0.0200 aloss:0.7626 eloss:-4.6721 aloss2:30.7485 exploreP:0.0323
Episode:1873 meanR:9.6700 R:10.0000 rate:0.0200 aloss:0.8108 eloss:-5.0453 aloss2:29.9737 exploreP:0.0323
Episode:1874 meanR:9.6800 R:10.0000 rate:0.0200 aloss:0.7681 eloss:-2.8580 aloss2:30.1227 exploreP:0.0323
Episode:1875 meanR:9.6900 R:10.0000 rate:0.0200 aloss:0.7488 eloss:-1.6630 aloss2:29.6393 exploreP:0.0322
Episode:1876 meanR:9.7000 R:10.0000 rate:0.0200 aloss:0.7160 eloss:-1.0576 aloss2:29.8384 exploreP:0.0322
Episode:1877 meanR:9.7000 R:10.0000 rate:0.0200 aloss:0.6749 eloss:0.3845 aloss2:29.7400 exploreP:0.0322
Episode:1878 meanR:9.6900 R:9.0000 rate:0.0180 aloss:0.8345 eloss:-1.8866 aloss2:29.4776 exploreP:0.0322
Episode:1879 meanR:9.7000 R:10.0000 rate:0.0200 

Episode:1948 meanR:9.9000 R:10.0000 rate:0.0200 aloss:0.8919 eloss:-4.6430 aloss2:28.6316 exploreP:0.0307
Episode:1949 meanR:9.9000 R:10.0000 rate:0.0200 aloss:0.8100 eloss:-0.5132 aloss2:29.1143 exploreP:0.0307
Episode:1950 meanR:9.8800 R:9.0000 rate:0.0180 aloss:0.8182 eloss:-1.4245 aloss2:28.8256 exploreP:0.0307
Episode:1951 meanR:9.8800 R:9.0000 rate:0.0180 aloss:0.8209 eloss:-3.5744 aloss2:30.0952 exploreP:0.0306
Episode:1952 meanR:9.8700 R:9.0000 rate:0.0180 aloss:0.8956 eloss:-2.9091 aloss2:28.3793 exploreP:0.0306
Episode:1953 meanR:9.8600 R:10.0000 rate:0.0200 aloss:0.8001 eloss:-2.2263 aloss2:29.9399 exploreP:0.0306
Episode:1954 meanR:9.8600 R:10.0000 rate:0.0200 aloss:0.8441 eloss:-3.8150 aloss2:29.0475 exploreP:0.0306
Episode:1955 meanR:9.8400 R:9.0000 rate:0.0180 aloss:0.8157 eloss:-1.0645 aloss2:30.0438 exploreP:0.0306
Episode:1956 meanR:9.8500 R:11.0000 rate:0.0220 aloss:0.7834 eloss:-4.0893 aloss2:29.4148 exploreP:0.0305
Episode:1957 meanR:9.8600 R:10.0000 rate:0.0200 al

Episode:2026 meanR:9.5900 R:10.0000 rate:0.0200 aloss:0.8090 eloss:-4.5476 aloss2:28.4678 exploreP:0.0292
Episode:2027 meanR:9.5900 R:9.0000 rate:0.0180 aloss:0.9327 eloss:2.7535 aloss2:27.3438 exploreP:0.0292
Episode:2028 meanR:9.5800 R:10.0000 rate:0.0200 aloss:0.9266 eloss:-4.2313 aloss2:27.4876 exploreP:0.0292
Episode:2029 meanR:9.5800 R:9.0000 rate:0.0180 aloss:0.7814 eloss:1.2003 aloss2:28.4488 exploreP:0.0292
Episode:2030 meanR:9.5800 R:10.0000 rate:0.0200 aloss:0.8107 eloss:-5.0700 aloss2:28.8122 exploreP:0.0291
Episode:2031 meanR:9.5800 R:10.0000 rate:0.0200 aloss:0.6923 eloss:-2.9352 aloss2:28.2336 exploreP:0.0291
Episode:2032 meanR:9.5800 R:10.0000 rate:0.0200 aloss:0.7753 eloss:-1.6708 aloss2:28.4485 exploreP:0.0291
Episode:2033 meanR:9.5800 R:9.0000 rate:0.0180 aloss:0.8145 eloss:-3.8646 aloss2:28.3818 exploreP:0.0291
Episode:2034 meanR:9.5900 R:10.0000 rate:0.0200 aloss:0.8596 eloss:-4.1129 aloss2:28.4071 exploreP:0.0291
Episode:2035 meanR:9.5900 R:9.0000 rate:0.0180 alos

Episode:2104 meanR:9.8200 R:10.0000 rate:0.0200 aloss:0.7653 eloss:-3.1092 aloss2:28.1447 exploreP:0.0278
Episode:2105 meanR:9.8100 R:10.0000 rate:0.0200 aloss:0.7977 eloss:0.2552 aloss2:28.0356 exploreP:0.0278
Episode:2106 meanR:9.8100 R:9.0000 rate:0.0180 aloss:0.8402 eloss:-4.7448 aloss2:29.0553 exploreP:0.0278
Episode:2107 meanR:9.8100 R:10.0000 rate:0.0200 aloss:0.7982 eloss:-2.6527 aloss2:28.0810 exploreP:0.0277
Episode:2108 meanR:9.8100 R:10.0000 rate:0.0200 aloss:0.7253 eloss:-3.0661 aloss2:28.9738 exploreP:0.0277
Episode:2109 meanR:9.8300 R:11.0000 rate:0.0220 aloss:0.7612 eloss:-2.5428 aloss2:28.3476 exploreP:0.0277
Episode:2110 meanR:9.8400 R:11.0000 rate:0.0220 aloss:0.7797 eloss:-3.6362 aloss2:27.6459 exploreP:0.0277
Episode:2111 meanR:9.8400 R:11.0000 rate:0.0220 aloss:0.7611 eloss:-2.5032 aloss2:28.1565 exploreP:0.0277
Episode:2112 meanR:9.8500 R:10.0000 rate:0.0200 aloss:0.7385 eloss:-2.3680 aloss2:27.6331 exploreP:0.0276
Episode:2113 meanR:9.8700 R:10.0000 rate:0.0200 

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
